## UseLib

In [96]:
############## 전처리 및 데이터 핸들링 ###############################

import numpy as np
import  pandas as pd
import os
from konlpy.tag import Okt 
okt = Okt()
from tqdm.notebook import tqdm
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


############## 머신러닝 모델#########################
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.svm import SVC
svc =  SVC()
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
import xgboost as xgb
from xgboost import XGBClassifier
xgc = XGBClassifier()


############## 평가 ###############################
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

## UseData

In [8]:
cd ~/WorkSpace/GoogleDrive/제주_전기차/data ## 경로이동

/home/miwoos/WorkSpace/GoogleDrive/제주_전기차/data


In [62]:
ls ## 데이터 위치 파악 및 데이터 리스트 확인

 elc_usewordcloud.png             '전기차 동호회 전기차크롤링.csv'
 elecafe.csv                       전기차_전처리및토큰화.csv
 lr.pkl                            전기차관련검색어인기순.csv
 lrc.pkl                           전기차관심도지역.csv
 tvect.pkl                         전기차관심증가.csv
 관련키워드.csv                    전기차전처리추가.csv
 네이버-제주전기차후기통합.csv     전기차키워드관련주제인기순.csv
 느영나영전기차크롤링.csv          전체본문의댓글결과_2022.03.06.xlsx
 느영나영전기차크롤링.csv.part     제사모크롤링.csv
'본문 검색 결과_2022.03.06.xlsx'   투플럭스sjeju.csv
 빈도수파악.png


In [11]:
df_train= pd.read_csv('전기차_전처리및토큰화.csv')
df_cafe = pd.read_csv('elecafe.csv')
df_Blog = pd.read_csv('네이버-제주전기차후기통합.csv')

## 러닝

### 널값 제거 및 전처리

In [63]:
df_train.head(2)

,전처리,평가
0,아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...,P
1,현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...,P


In [72]:
print(df_train.isnull().sum())
print(df_cafe.isnull().sum())
print(df_Blog.isnull().sum())

전처리    0
평가     0
dtype: int64
제목      0
내용      0
댓글      0
유형      0
조회수     0
작성날짜    0
월별      0
카페명     0
년별      0
dtype: int64
제목      0
날짜      0
내용      0
href    0
dtype: int64


In [21]:
del df_Blog['index'] # 인덱스칼럼이 널값이많아서 컬럼 자체를 없애 버림 
df_train.dropna(inplace=True) # 드랍으로 널값제거

### 이진분류

In [33]:
df_이진분류 = copy.deepcopy(df_train[df_train !='E'])

In [76]:
df_이진분류.head(2)
df_이진분류.dropna(inplace=True)
df_이진분류.isnull().sum()

전처리    0
평가     0
dtype: int64

In [77]:
X_train,X_test,y_train,y_test = train_test_split (
    df_이진분류['전처리'],df_이진분류['평가'],test_size=0.25,random_state=2021
)

In [78]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((218401,), (72801,), (218401,), (72801,))

##### 타겟값 라벨링

In [79]:
y_train_label = LabelEncoder().fit_transform(y_train)
y_test_label = LabelEncoder().fit_transform(y_test)

In [80]:
y_train_label,y_test_label

(array([1, 0, 1, ..., 0, 1, 0]), array([1, 1, 0, ..., 1, 1, 1]))

#### CV binary

In [98]:
cvect_binary =  CountVectorizer()
cvect_binary.fit(X_train)

CountVectorizer()

In [85]:
X_train_cv_binary = cvect_binary.transform(X_train)
X_test_cv_binary =  cvect_binary.transform(X_test)

#### TF-IDF binary

In [86]:
tvect_binary = TfidfVectorizer(ngram_range=(1,2), max_df = 0.9)

In [87]:
%time tvect_binary.fit(X_train)


CPU times: user 2min 53s, sys: 696 ms, total: 2min 53s
Wall time: 2min 53s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2))

In [101]:
%time X_train_tv = tvect_binary.transform(X_train)
%time X_test_tv = tvect_binary.transform(X_test)

CPU times: user 16.1 s, sys: 123 ms, total: 16.3 s
Wall time: 16.3 s
CPU times: user 5.22 s, sys: 23.7 ms, total: 5.24 s
Wall time: 5.26 s


### 다중분류

In [89]:
XX_train,XX_test,yy_train,yy_test = train_test_split (
    df_train['전처리'],df_train['평가'],test_size=0.25,random_state=2021
)

In [90]:
XX_train.shape,XX_test.shape,yy_train.shape,yy_test.shape

((296083,), (98695,), (296083,), (98695,))

#### cv 

In [97]:
cvect  = copy.deepcopy(CountVectorizer())
cvect.fit(XX_train)

CountVectorizer()

In [ ]:
X_train_cv = cvect.transform(XX_train)
X_test_cv=  cvect.transform(XX_test)

#### TF-IDF

In [99]:
tvect = copy.deepcopy(TfidfVectorizer(ngram_range=(1,2), max_df = 0.9))

In [100]:
%time tvect.fit(XX_train)
%time XX_train_tv = tvect.transform(XX_train)
%time XX_test_v = tvect.transform(XX_test)


CPU times: user 3min 8s, sys: 844 ms, total: 3min 9s
Wall time: 3min 8s
CPU times: user 18.4 s, sys: 147 ms, total: 18.5 s
Wall time: 18.6 s
CPU times: user 5.86 s, sys: 35.8 ms, total: 5.9 s
Wall time: 5.9 s


## 모델 비교